In [21]:
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sb
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
from scipy.stats import binomtest
#from scipy.stats import chisquare, binomtest
#from sklearn.linear_model import LinearRegression
#import statsmodels.api as sm
#import warnings
#warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
#from sklearn.exceptions import ConvergenceWarning
#warnings.simplefilter("ignore", category=ConvergenceWarning)

In [3]:
dir_data = '../data'
dir_out = 'outputs'

In [4]:
### read methylation data and sample order files
file_in = f'{dir_data}/methyl_data.tsv'
df = pd.read_table(file_in, sep='\t', header=0, keep_default_na=False)
df = df.loc[df.region_id.str.contains('_5UTR')].copy()
print('df:')
display(df)

file_sam = f'{dir_data}/methyl_data_sample_order.txt'
with open(file_sam, 'r') as fh:
    sample_order = fh.readline().split(',')[:-1] # last one is an empty string
print(f'number of samples: {len(sample_order)}')

df:


,chrom,pos,end,region_id,methyl_h1,methyl_h2,methyl_h0,bp_dev_h1,bp_dev_h2,bp_dev_h0,rn_h1,rn_h2,rn_h0,ps_h1,ps_h2
1,chrX,147911920,147912179,FMR1_5UTR,.;.;.;91|95|100|99|96|100|99|98|98|96|96|97|94...,.;.;.;64|20|7|2|0|0|0|0|0|0|2|0|1|0|0|0|0|0|3|...,.;0|0|0|0|0|0|0|1|2|32|27|13|7|5|46|27|29|15|3...,".;.;.;32;.;.;30;.;31,30;34;27,27,27,29;.;.;.;3...",".;.;.;31,43,30;.;.;30,28,28,31;.;30,30,32;34,2...",".;51,51,51,57,51,51,52;30;.;.;30;.;34,31;.;.;....",.;.;.;m54340U_200716_153006/27724288/ccs;.;.;m...,".;.;.;m54340U_200716_153006/75497573/ccs,m5434...",".;m54340U_210523_044800/52758275/ccs,m54340U_2...",".;.;.;10185;.;.;10215;.;10215,10215;10185;1018...",".;.;.;10185,10185,10185;.;.;10215,10215,10215,..."
7,chrX,149505139,149505306,IDS_5UTR,.;.;.;8|2|2|1|0|0|0|0|0|1|0|7|1|0|3|2|1|4|0|0|...,.;.;.;62|51|80|91|75|95|89|98|99|98|97|99|99|9...,.;0|2|0|1|0|1|6|2|1|8|9|13|6|3|4|8|5|0|1|0|0|0...,".;.;.;1;.;.;.;.;.;2;.;.;.;.;0,0,2;.;.;-1,1,0;....",".;.;.;0;.;.;.;.;.;.;.;.;.;.;.;.;.;1,0,1;.;.;.;...",".;0,2;3;.;0,0;0;-1,1,0,1;0,-2,0;0,0,-1,0;0,0,1...",.;.;.;m54340U_200716_153006/128059025/ccs;.;.;...,.;.;.;m54340U_200716_153006/133105829/ccs;.;.;...,".;m54340U_200806_070905/88277013/ccs,m54340U_2...",".;.;.;10185;.;.;.;.;.;10185;.;.;.;.;10185,1018...",".;.;.;10185;.;.;.;.;.;.;.;.;.;.;.;.;.;10185,10..."
8,chrX,149631582,149631792,TMEM185A_5UTR,.;.;.;.;.;.;95|91|35|70|18|12|12|62|41|61|26|6...,.;.;.;.;73|60|29|4|2|7|3|1|1|1|1|0|0|0|8|17|11...,5|0|0|0|0|13|5|7|20|56|43|61|72|96|91|64|21|11...,".;.;.;.;.;.;-21,-21,-21;.;-21,-15;.;15,9;.;0;....",".;.;.;.;-21;.;.,9,0;.;-21,-21;.;-21;.;-11;.;.;...","-21,-19;-21,-21,-21,-21,-18,-21,-21,-25;.;-19;...",.;.;.;.;.;.;m54340U_210703_193517/110166254/cc...,.;.;.;.;m54340U_200716_153006/94110941/ccs;.;m...,"m54340U_200716_001732/174262380/ccs,m54340U_20...",".;.;.;.;.;.;10215,10215,10215;.;10215,10215;.;...",".;.;.;.;10185;.;10215,10215,10215;.;10215,1021..."
9,chrX,150363318,150363530,MAMLD1_5UTR,.;.;.;3|2|5|5|4|1|3|3|1|0|0|0|0|0|0|0|0|0|0|4|...,.;.;.;.;.;.;71|65|46|44|7|4|0|0|0|0|0|2|64|66|...,0|2|2|2|0|0|1|6|1|3|2|49|71|55|13|6|11|26|7|5|...,".;.;.;0;.;.;-1;.;0,0;32,33,35,29,29,34,34;1,-1...",".;.;.;.;.;.;1,3;.;.;1,2,2;0,1,3;.;1,1;.;1,0;.;...","-1;0,0,0;.;.;0,3;4,1;1,0,1,1;0,1,0,0;.;.;.;37,...",.;.;.;m54340U_200716_153006/71501671/ccs;.;.;m...,.;.;.;.;.;.;m54340U_200806_223416/98175724/ccs...,m54340U_200716_001732/24969595/ccs;m54340U_210...,".;.;.;10185;.;.;10215;.;10215,10215;10185,1018...",".;.;.;.;.;.;10215,10215;.;.;10185,10185,10185;..."
10,chrX,150568622,150568662,MTM1_5UTR,".;.;.;.;.;.;.;.;.;3|13|4|2|0|0,98|99|98|95|97|...",".;.;.;.;.;.;.;.;.;100|100|100|100|100|100,45|5...","1|0|0|0|0|0,2|4|2|0|0|6,7|14|13|9|1|0,2|13|6|0...",".;.;.;.;.;.;.;.;.;0,0;0,0;.;0,0,0;.;.;.;.;0,0;...",".;.;.;.;.;.;.;.;.;0,2;0;.;0,-1;.;.;.;.;1;.;.;....","0,0,0,0,0;0,0,0;0;.;0,0;.;2,0,0,0;.;.,0,0,1,0,...",.;.;.;.;.;.;.;.;.;m54340U_200611_224317/158271...,.;.;.;.;.;.;.;.;.;m54340U_200611_224317/107350...,"m54340U_200716_001732/162859070/ccs,m54340U_20...",".;.;.;.;.;.;.;.;.;10185,10185;10185,10185;.;10...",".;.;.;.;.;.;.;.;.;10185,10185;10185;.;10185,10..."
11,chrX,150693397,150693530,MTMR1_5UTR,.;.;.;.;.;.;.;.;.;2|1|2|10|11|5|3|2|3|1|1|5|18...,.;.;.;.;.;.;.;.;.;.;.;.;2|0|0|5|2|5|12|23|34|4...,6|4|6|5|10|63|82|90|76|34|11|6|1|2|2|1|1|3|2|4...,".;.;.;.;.;.;.;.;.;1,0,1,0,2,0;1,2,2,1,1;.;.;.;...",".;.;.;.;.;.;.;.;.;.;.;.;-1,1,-1;.;0,2,3;.;.;0,...","1,0,0;0,0,0,0,2,0,1;.;0;0,0;0;0,0,0,1,1;0,-1,0...",.;.;.;.;.;.;.;.;.;m54340U_200611_224317/609751...,.;.;.;.;.;.;.;.;.;.;.;.;m54340U_200611_224317/...,"m54340U_200716_001732/53740679/ccs,m54340U_200...",".;.;.;.;.;.;.;.;.;10185,10185,10185,10185,1018...",".;.;.;.;.;.;.;.;.;.;.;.;10185,10185,10185;.;10..."
12,chrX,150898590,150898668,CD99L2_5UTR,.;.;.;8|7|9|52|43|81|48|25;.;.;88|47|83|82|98|...,".;.;.;2|0|0|1|0|0|0|0;.;.;0|0|0|0|0|0|0|13,62|...","0|0|1|0|1|1|6|4,1|1|1|4|3|16|46|69;0|0|0|0|1|1...",".;.;.;1;.;.;0,1;.;0;0,0,-1;0,0;.;0,0,1;.;.;.;....",".;.;.;1;.;.;0,0,0;.;.;0,0,1;2,

number of samples: 270


In [5]:
### expand columns for each sample, because each column consists of semicolon-separated sample values
df_exnd = df[['chrom' ,'pos', 'end', 'region_id']].copy()
cols = df.columns.tolist()[4:]
#print(cols)

for idx, sample in enumerate(sample_order):
    for col in cols:
        df_exnd[f'{sample}_{col}'] = df[col].str.split(';').apply(lambda row: row[idx])
display(df_exnd)

,chrom,pos,end,region_id,REACH000026_methyl_h1,REACH000026_methyl_h2,REACH000026_methyl_h0,REACH000026_bp_dev_h1,REACH000026_bp_dev_h2,REACH000026_bp_dev_h0,...,REACH000685_methyl_h2,REACH000685_methyl_h0,REACH000685_bp_dev_h1,REACH000685_bp_dev_h2,REACH000685_bp_dev_h0,REACH000685_rn_h1,REACH000685_rn_h2,REACH000685_rn_h0,REACH000685_ps_h1,REACH000685_ps_h2
1,chrX,147911920,147912179,FMR1_5UTR,.,.,.,.,.,.,...,.,2|0|15|0|2|4|35|71|4|0|1|7|0|10|0|2|0|0|0|0|58...,.,.,"13,17,33,29,16",.,.,"658567b8-0bb0-48a8-a85d-619f26f01f60,ec6386b7-...",.,.
7,chrX,149505139,149505306,IDS_5UTR,.,.,.,.,.,.,...,.,11|0|0|0|0|15|0|0|0|4|89|91|2|0|0|0|1|9|93|89|...,.,.,"6,-14,-3,-4,-29,-24,-17,-8,-69",.,.,"dbdc51c7-868d-4c1e-aef8-54b4be7616b1,ee73bd80-...",.,.
8,chrX,149631582,149631792,TMEM185A_5UTR,.,.,5|0|0|0|0|13|5|7|20|56|43|61|72|96|91|64|21|11...,.,.,"-21,-19",...,.,"100|4|30|7|40|97|0,0|0|0|0|2|0|0|1|0|0|0|0|2|2...",.,.,"-23,-13,-2,-8,-21,.",.,.,"0bdcf4ce-425a-4468-9796-785e0a2817f8,8d6dac54-...",.,.
9,chrX,150363318,150363530,MAMLD1_5UTR,.,.,0|2|2|2|0|0|1|6|1|3|2|49|71|55|13|6|11|26|7|5|...,.,.,-1,...,.,1|0|1|0|0|0|4|0|3|61|9|12|46|79|84|75|0|1|0|0|...,.,.,"-8,-3,-11,-2,-17,8,-5,-2",.,.,"b0ddb187-c71b-4f31-8e70-da57f39b8cff,9b1c13e4-...",.,.
10,chrX,150568622,150568662,MTM1_5UTR,.,.,"1|0|0|0|0|0,2|4|2|0|0|6,7|14|13|9|1|0,2|13|6|0...",.,.,"0,0,0,0,0",...,.,"40|36|9|41|79|30|22|2|45,4|2|0|0|0|0|5,0|4|0|1...",.,.,"9,-1,-2,2,6",.,.,"7178be63-b143-43d2-b646-7f250eee984a,3152af87-...",.,.
11,chrX,150693397,150693530,MTMR1_5UTR,.,.,6|4|6|5|10|63|82|90|76|34|11|6|1|2|2|1|1|3|2|4...,.,.,"1,0,0",...,.,"0|9|0|1|0|0|0|1|13|0|16|0|0|2|0|5,55|70|9|2|1|...",.,.,"-10,-2,5,0,-3,3,5,-5,-16,-4",.,.,"33acaac3-8252-404b-b9e8-e6c5fde090e9,21350c97-...",.,.
12,chrX,150898590,150898668,CD99L2_5UTR,.,.,"0|0|1|0|1|1|6|4,1|1|1|4|3|16|46|69",.,.,"0,0",...,"1|0|0|7|7|0|0|0|2,45|0|21|0|0|2|0|2","46|85,0|49|2|0|0|4|0|0,0|80|0|2|0|3|0|36,10|0",-3,"4,0","-6,-6,-5,.",5af74e3f-fde8-469d-9c44-8ca2cce771d2,"34a1e889-e5f6-4400-8f70-0d32c5afd4d6,7d1b38e0-...","1ae72adc-1d37-4141-bfa6-590e0ec211d1,46f0b406-...",10185,"10185,10185"
13,chrX,150983336,150983376,HMGB3_5UTR,.,.,.,.,.,.,...,.,"0|0|12|0,0|0|0|0|2",.,.,"-1,-6",.,.,"519f9439-750b-421f-9828-4705fb07f845,94933348-...",.,.


In [9]:
### Get coverage information for each sample
file_cov = f'{dir_data}/coverages_mod.tsv'
df_cov = pd.read_table(file_cov, header=0, sep='\t')
multiplat_samples = ['REACH000236', 'REACH000530', 'REACH000531', 'REACH000532', 'REACH000436']
df_cov['SAMPLE'] = df_cov.apply(lambda row: row['SAMPLE']+'_'+row['COHORT'] if row['SAMPLE'] in multiplat_samples else row['SAMPLE'], axis=1)
display(df_cov)

def get_plat(sample):
    try:
        ret = df_cov[df_cov.SAMPLE == sample]['COHORT'].values[0]
    except:
        print(f'problem with samples: {sample}')
        ret = ''
    return ret

def get_cov(sample):
    try:
        ret = df_cov[df_cov.SAMPLE == sample]['MEAN_COVERAGE'].values[0]
    except:
        print(f'problem with samples: {sample} in get_cov')
        ret = ''
    return ret

plat_dict = {sample: get_plat(sample) for sample in df_cov.SAMPLE.tolist()}
plat_dict['REACH000226_blood'] = plat_dict['REACH000226']
plat_dict['REACH000226_sperm'] = plat_dict['REACH000226']
#print('plat_dict:')
#print(plat_dict)

cov_dict = {sample: get_cov(sample) for sample in df_cov.SAMPLE.tolist()}
#print('cov_dict:')
#print(cov_dict)

,SAMPLE,COHORT,MEAN_COVERAGE
0,HG004,ONT,81.44
1,REACH000626,ONT,10.51
2,REACH000546,ONT,10.87
3,REACH000683,ONT,9.97
4,REACH000563,ONT,8.68
...,...,...,...
283,REACH000284,PB,3.43
284,REACH000672,PB,5.02
285,REACH000292,PB,9.00
286,REACH000066,PB,1.61


In [11]:
### Get sample relation information from a psam file
file_psam = f'{dir_data}/REACH_LR_platform.psam'
df_p = pd.read_table(file_psam, sep='\t', header=None, names = ['famid', 'sample_id', 'dad', 'mom', 'sex', 'phen'])
#print('df_p:')
#display(df_p)
ds_sex = df_p.set_index('sample_id').sex
#display(ds_sex)

# make mom and dad dictionary
sample_dad_dict = {}
sample_mom_dict = {}
for sample, dad, mom in zip(df_p.sample_id, df_p.dad, df_p.mom):
    #print(sample, dad, mom)
    sample_dad_dict[sample] = dad
    sample_mom_dict[sample] = mom

#print('complete trios:')
df_trios = df_p.loc[df_p.dad.isin(df_p.sample_id) & df_p.mom.isin(df_p.sample_id)]
print('df_trios:')
display(df_trios.shape)

### make a samples dataframe containing all samples in complete trios
df_samples = pd.DataFrame()
df_samples['sample_id'] = df_trios['sample_id']
df_samples['rel'] = 'C'

df_dads = pd.DataFrame()
df_dads['sample_id'] =  list(set(df_trios['dad'].tolist()))
df_dads['rel'] = 'D'
df_samples = pd.concat([df_samples, df_dads])

df_moms = pd.DataFrame()
df_moms['sample_id'] =  list(set(df_trios['mom'].tolist()))
df_moms['rel'] = 'M'
df_samples = pd.concat([df_samples, df_moms])

df_samples['sex'] = df_samples.sample_id.map(ds_sex)
df_samples['plat'] = df_samples.sample_id.map(plat_dict)
df_samples['coverage'] = df_samples.sample_id.map(cov_dict)
### add blood/sperm sections for REACH000226
row_add = pd.DataFrame({'sample_id':['REACH000226_blood', 'REACH000226_sperm'] ,'rel':['D', 'D'] ,'sex':[1, 1] ,'plat':['ONT', 'ONT'] ,'coverage':[1.92, 27.87]})
df_samples = pd.concat([df_samples, row_add])
pd.set_option('display.max_rows', 10)
print('df_samples:')
display(df_samples)

df_trios:


(117, 6)

df_samples:


,sample_id,rel,sex,plat,coverage
10,REACH000026,C,1,PB,3.22
11,REACH000058,C,1,PB,6.19
12,REACH000065,C,1,PB,1.22
15,REACH000086,C,1,PB,2.70
18,REACH000089,C,2,PB,3.27
...,...,...,...,...,...
60,REACH000672,M,2,PB,5.02
61,REACH000195,M,2,PB,2.67
62,REACH000651,M,2,PB,5.75
0,REACH000226_blood,D,1,ONT,1.92


In [14]:
### Read meta data of subjects
meta_file = f'{dir_data}/REACH_sample_info.tsv'
df_meta = pd.read_table(meta_file, sep='\t', header=0)

rename_samples_dict = {'REACH000236': 'REACH000236_PB', 
                       'REACH000436': 'REACH000436_PB', 
                       'REACH000530': 'REACH000530_PB', 
                       'REACH000531': 'REACH000531_ONT', 
                       'REACH000532': 'REACH000532_ONT'}
df_meta['Sample_ID_orig'] = df_meta['Sample_ID']
df_meta['Sample_ID'] = df_meta.apply(lambda row: rename_samples_dict[row['Sample_ID']] 
                                     if row['Sample_ID'] in rename_samples_dict else row['Sample_ID'], axis=1)

aff_dict = {}
for sample, aff in zip(df_meta['Sample_ID'].tolist(), df_meta['Affected'].tolist()):
    aff_dict[sample] = aff

In [15]:
### Read PCA data of subjects
file_pca = f'{dir_data}/master_phen_4.SBayesR.20210915.csv'

df_pca = pd.read_table(file_pca, sep=',', header=0)
cols = ['fid', 'iid', 'phen','sex','cohort','duo','family','is_eur','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10', 
        'PC1_PAT','PC2_PAT','PC3_PAT','PC4_PAT','PC5_PAT','PC6_PAT','PC7_PAT','PC8_PAT','PC9_PAT','PC10_PAT', 
        'PC1_MAT','PC2_MAT','PC3_MAT','PC4_MAT','PC5_MAT','PC6_MAT','PC7_MAT','PC8_MAT','PC9_MAT','PC10_MAT']
df_pca = df_pca[cols]

rename_samples_dict = {'REACH000236': 'REACH000236_PB', 
                       'REACH000436': 'REACH000436_PB', 
                       'REACH000530': 'REACH000530_PB', 
                       'REACH000531': 'REACH000531_ONT', 
                       'REACH000532': 'REACH000532_ONT'}
df_pca['iid'] = df_pca.apply(lambda row: rename_samples_dict[row['iid']] if row['iid'] in rename_samples_dict else row['iid'], axis=1)
#display(df_pca)

# first filter for the kids, then add parents to them
df_pca_flt = df_pca.loc[df_pca.iid.isin(df_trios.sample_id)].copy()
df_pca_flt['rel'] = 'C'
#print('df_pca_flt:')
#display(df_pca_flt)

df_parents = pd.DataFrame()
for index, row in df_pca_flt.iterrows():
    sample = row['iid']
    fid = row['fid']
    phen = row['phen']
    is_eur = row['is_eur']
    pc_mat = [row[f'PC{x}_MAT'] for x in range(1,11)]
    pc_pat = [row[f'PC{x}_PAT'] for x in range(1,11)]
    dad = sample_dad_dict[sample]
    mom = sample_mom_dict[sample]
    
    dict_mat = {'fid': [fid], 'iid': [mom], 'phen': ['.'], 'sex': ['Female'], 'cohort': [1], 'duo': ['.'], 'is_eur': [is_eur]}
    dict_mat.update({f'PC{x}': [pc_mat[x-1]] for x in range(1,11)})
    df_mat = pd.DataFrame(dict_mat)

    dict_pat = {'fid': [fid], 'iid': [dad], 'phen': ['.'], 'sex': ['Male'], 'cohort': [1], 'duo': ['.'], 'is_eur': [is_eur]}
    dict_pat.update({f'PC{x}': [pc_pat[x-1]] for x in range(1,11)})
    df_pat = pd.DataFrame(dict_pat)
    
    if df_parents.shape[0] == 0 or mom not in df_parents.iid.tolist():
        df_parents = pd.concat([df_parents, df_mat])
    if df_parents.shape[0] == 0 or dad not in df_parents.iid.tolist():
        df_parents = pd.concat([df_parents, df_pat])

df_parents['rel'] = 'P'
#print('df_parents:')
#display(df_parents)

df_pca_flt = pd.concat([df_pca_flt, df_parents])

df_pca_flt = pd.merge(df_pca_flt, df_meta[['Sample_ID', 'Affected', 'Affected_ASD']], how='inner', left_on='iid', right_on='Sample_ID')
df_pca_flt = pd.merge(df_pca_flt, df_cov[['SAMPLE', 'MEAN_COVERAGE']], how='inner', left_on='iid', right_on='SAMPLE')

df_pca_flt['Phenotype'] = df_pca_flt.apply(lambda row: 1 if row.Affected == 'Yes' else 0, axis=1)
#df_pca_flt['Phenotype_ASD'] = df_pca_flt.apply(lambda row: 1 if row.Affected_ASD == 'Yes' else 0, axis=1)
df_pca_flt['Platform'] = df_pca_flt.apply(lambda row: get_plat(row.iid), axis=1)

### make a original iid w/o platform for the handfull of samples
df_pca_flt['iid_orig'] = df_pca_flt.iid.str.split('_').str[0]

print('df_pca_flt:')
display(df_pca_flt)

df_pca_flt:


,fid,iid,phen,sex,cohort,duo,family,is_eur,PC1,PC2,...,PC10_MAT,rel,Sample_ID,Affected,Affected_ASD,SAMPLE,MEAN_COVERAGE,Phenotype,Platform,iid_orig
0,F0001,REACH000001,ASD,Male,1,trio,1.0,1,0.008627,-0.005607,...,-0.014983,C,REACH000001,Yes,Yes,REACH000001,10.15,1,ONT,REACH000001
1,F0026,REACH000026,ASD,Male,1,trio,1.0,0,0.095364,-0.045230,...,-0.011925,C,REACH000026,Yes,Yes,REACH000026,3.22,1,PB,REACH000026
2,F0058,REACH000058,ASD,Male,1,trio,1.0,1,0.002422,-0.009216,...,0.016383,C,REACH000058,Yes,Yes,REACH000058,6.19,1,PB,REACH000058
3,F0065,REACH000065,ASD,Male,1,trio,1.0,0,0.252774,0.107933,...,-0.009906,C,REACH000065,Yes,Yes,REACH000065,1.22,1,PB,REACH000065
4,F0078,REACH000086,ASD,Male,1,trio,2.0,0,0.044785,0.038065,...,-0.018303,C,REACH000086,Yes,Yes,REACH000086,2.70,1,PB,REACH000086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,F0288,REACH000756,.,Male,1,.,NaN,0,0.204041,0.127559,...,NaN,P,REACH000756,No,No,REACH000756,1.55,0,PB,REACH000756
239,F0290,REACH000763,.,Female,1,.,NaN,1,-0.030630,0.003052,...,NaN,P,REACH000763,No,No,REACH000763,3.36,0,PB,REACH000763
240,F0290,REACH000764,.,Male,1,.,NaN,1,-0.027327,-0.004878,...,NaN,P,REACH000764,No,No,REACH000764,1.20,0,PB,REACH000764
241,F0291,REACH000768,.,Female,1,.,NaN,0,0.091442,-0.016917,...,NaN,P,REACH000768,No,No,REACH000768,0.10,0,PB,REACH000768


In [16]:
### make a table of raw data.
pd.set_option('display.max_columns', None)
region_id = 'FMR1_5UTR'
xci_region_ids = ['IDS_5UTR', 'TMEM185A_5UTR', 'MAMLD1_5UTR', 'MTM1_5UTR', 'MTMR1_5UTR', 'CD99L2_5UTR', 'HMGB3_5UTR']
df_methyl = pd.DataFrame()

this_idx = df_exnd.loc[df_exnd.region_id==region_id].index.tolist()[0]
cols_bp = [f'bp_dev_h1', f'bp_dev_h2', f'bp_dev_h0']
cols_met = [f'methyl_h1', f'methyl_h2', f'methyl_h0']
for col in cols_bp+cols_met:
    col_out = f'xci_read_mean_methyl_lik_{col.split("_")[-1]}'
    df_methyl[col] = pd.Series(dtype='str')
    df_methyl[col_out] = pd.Series(dtype='str')

for sample in sample_order:
    
    ### set bp deviation columns
    for col in cols_bp:
        df_methyl.loc[sample, col] = df_exnd.loc[df_exnd.region_id==region_id].loc[this_idx, f'{sample}_{col}']
    
    ### set FMR1 methylation columns
    for col in cols_met:
        df_methyl.loc[sample, col] = df_exnd.loc[df_exnd.region_id==region_id].loc[this_idx, f'{sample}_{col}']

    ### set 7 XCI gene methylation columns
    for col in cols_met:
        col_out = f'xci_read_mean_methyl_lik_{col.split("_")[-1]}'
        read_met_list = [y for x in df_exnd.loc[df_exnd.region_id.isin(xci_region_ids)][f'{sample}_{col}'].tolist() for y in x.split(',') if y!='.']
        df_methyl.loc[sample, col_out] = ','.join([str(round(np.mean([float(xx) for xx in x.split('|')]))) for x in read_met_list])

### correct phasing of REACH000563 and REACH000561 manually due to poor phasing
bp_dev_thr = 60
if ('REACH000563' in df_methyl.index):
    this_sample = 'REACH000563'
    # h1 and h2 are empty and we want to make them from h0 based on bp deviation
    df_methyl.loc[this_sample, 'bp_dev_h2'] = ','.join([x for x in df_methyl.loc[this_sample, 'bp_dev_h0'].split(',') if x != '.' and float(x)>bp_dev_thr])
    df_methyl.loc[this_sample, 'bp_dev_h1'] = ','.join([x for x in df_methyl.loc[this_sample, 'bp_dev_h0'].split(',') if x != '.' and float(x)<bp_dev_thr])
    df_methyl.loc[this_sample, 'methyl_h2'] = ','.join([m for b, m in zip(df_methyl.loc[this_sample, 'bp_dev_h0'].split(','),  
                                                                          df_methyl.loc[this_sample, 'methyl_h0'].split(',')) if b != '.' and float(b)>bp_dev_thr])
    df_methyl.loc[this_sample, 'methyl_h1'] = ','.join([m for b, m in zip(df_methyl.loc[this_sample, 'bp_dev_h0'].split(','),  
                                                                          df_methyl.loc[this_sample, 'methyl_h0'].split(',')) if b != '.' and float(b)<bp_dev_thr])
    df_methyl.loc[this_sample, 'methyl_h0'] = '.'
    df_methyl.loc[this_sample, 'bp_dev_h0'] = '.'

if ('REACH000561' in df_methyl.index):
    this_sample = 'REACH000561'
    ### first fix h2: bring one read from h1
    # one read in h1 has bp=83 and belongs to h2. we want to bring it in h2
    df_methyl.loc[this_sample, 'bp_dev_h2'] = ','.join(df_methyl.loc[this_sample, 'bp_dev_h2'].split(',') + 
                                               [x for x in df_methyl.loc[this_sample, 'bp_dev_h1'].split(',') if x != '.' and float(x)>bp_dev_thr])
    df_methyl.loc[this_sample, 'methyl_h2'] = ','.join(df_methyl.loc[this_sample, 'methyl_h2'].split(',') + 
                                                       [m for b, m in zip(df_methyl.loc[this_sample, 'bp_dev_h1'].split(','), 
                                                                          df_methyl.loc[this_sample, 'methyl_h1'].split(',')) if b != '.' and float(b)>bp_dev_thr])
    df_methyl.loc[this_sample, 'bp_dev_h1'] = ','.join([x for x in df_methyl.loc[this_sample, 'bp_dev_h1'].split(',') if x != '.' and float(x)<bp_dev_thr])
    df_methyl.loc[this_sample, 'methyl_h1'] = ','.join([m for b, m in zip(df_methyl.loc[this_sample, 'bp_dev_h1'].split(','),  
                                                                          df_methyl.loc[this_sample, 'methyl_h1'].split(',')) if b != '.' and float(b)<bp_dev_thr])

### compute repeat length based on bp deviation
for col in cols_bp:
    col_out = f'repeat_length_{col.split("_")[-1]}'
    df_methyl[col_out] = df_methyl[col].apply(lambda x: ','.join([str(round(float(xx) / 3) + 20) if xx!='.' else '.' for xx in x.split(',')]))
    
### take average of repeat length
cols_rep_len = [f'repeat_length_h1', f'repeat_length_h2', f'repeat_length_h0']
for col in cols_rep_len:
    df_methyl[f'mean_{col}'] = df_methyl.apply(lambda row: np.mean([float(x) for x in row[col].split(',') if x != '.']) 
                                                 if len([x for x in row[col].split(',') if x != '.'])>0 else '.', axis=1)

### compute mean methylation likelihood for each read
for col in cols_met:
    df_methyl[f'read_mean_methyl_lik_{col.split("_")[-1]}'] = df_methyl.apply(lambda row: ','.join([str(round(np.mean([float(xx) for xx in x.split('|')]))) for x in row[col].split(',') if x != '.'])
                                                                             if row[col] != '.' else '.', axis=1)
### set number of methylation/unmethylation reads
for col in cols_met:
    hp = col.split("_")[-1]
    df_methyl[f'num_methyl_reads_{hp}'] = df_methyl[f'read_mean_methyl_lik_{hp}'].apply(lambda x: len([xx for xx in x.split(',') if float(xx)>=50]) if x != '.' else '.')
    df_methyl[f'num_unmethyl_reads_{hp}'] = df_methyl[f'read_mean_methyl_lik_{hp}'].apply(lambda x: len([xx for xx in x.split(',') if float(xx)<50]) if x != '.' else '.')
    df_methyl[f'frac_hap_methyl_{hp}'] = df_methyl.apply(lambda row: int(row[f'num_methyl_reads_{hp}'])/(int(row[f'num_methyl_reads_{hp}'])+int(row[f'num_unmethyl_reads_{hp}'])) 
                                                           if (row[f'num_methyl_reads_{hp}']!='.' and row[f'num_unmethyl_reads_{hp}']!='.') else '.', axis=1)

### set 7 XCI gene number of methylation/unmethylation reads
for col in cols_met:
    hp = col.split("_")[-1]
    df_methyl[f'xci_num_methyl_reads_{hp}'] = df_methyl[f'xci_read_mean_methyl_lik_{hp}'].apply(lambda x: len([xx for xx in x.split(',') if float(xx)>=50]) if (x != '.' and x != '') else '.')
    df_methyl[f'xci_num_unmethyl_reads_{hp}'] = df_methyl[f'xci_read_mean_methyl_lik_{hp}'].apply(lambda x: len([xx for xx in x.split(',') if float(xx)<50]) if (x != '.' and x != '') else '.')
    df_methyl[f'xci_frac_hap_methyl_{hp}'] = df_methyl.apply(lambda row: int(row[f'xci_num_methyl_reads_{hp}'])/(int(row[f'xci_num_methyl_reads_{hp}'])+int(row[f'xci_num_unmethyl_reads_{hp}'])) 
                                                           if (row[f'xci_num_methyl_reads_{hp}']!='.' and row[f'xci_num_unmethyl_reads_{hp}']!='.') else '.', axis=1)


display(df_methyl)

,bp_dev_h1,xci_read_mean_methyl_lik_h1,bp_dev_h2,xci_read_mean_methyl_lik_h2,bp_dev_h0,xci_read_mean_methyl_lik_h0,methyl_h1,methyl_h2,methyl_h0,repeat_length_h1,repeat_length_h2,repeat_length_h0,mean_repeat_length_h1,mean_repeat_length_h2,mean_repeat_length_h0,read_mean_methyl_lik_h1,read_mean_methyl_lik_h2,read_mean_methyl_lik_h0,num_methyl_reads_h1,num_unmethyl_reads_h1,frac_hap_methyl_h1,num_methyl_reads_h2,num_unmethyl_reads_h2,frac_hap_methyl_h2,num_methyl_reads_h0,num_unmethyl_reads_h0,frac_hap_methyl_h0,xci_num_methyl_reads_h1,xci_num_unmethyl_reads_h1,xci_frac_hap_methyl_h1,xci_num_methyl_reads_h2,xci_num_unmethyl_reads_h2,xci_frac_hap_methyl_h2,xci_num_methyl_reads_h0,xci_num_unmethyl_reads_h0,xci_frac_hap_methyl_h0
REACH000026,.,,.,,.,"22,10,10,0,2,7,4,2,20,19,20,2,18",.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,0,13,0.0
REACH000058,.,,.,,"51,51,51,57,51,51,52","3,4,9,10,2,2,7,2,39,8,12,1,17,0,0,1,1,14,29,10...",.,.,0|0|0|0|0|0|0|1|2|32|27|13|7|5|46|27|29|15|3|4...,.,.,"37,37,37,39,37,37,37",.,.,37.285714,.,.,"8,9,11,17,23,5,17",.,.,.,.,.,.,0,7,0.0,.,.,.,.,.,.,0,28,0.0
REACH000065,.,,.,,30,"9,10",.,.,62|37|36|51|8|2|0|0|0|0|0|7|46|88|91|87|84|66|...,.,.,30,.,.,30.0,.,.,22,.,.,.,.,.,.,0,1,0.0,.,.,.,.,.,.,0,2,0.0
REACH000066,32,"1,12,34","31,43,30","87,0",.,"71,77,1,88",91|95|100|99|96|100|99|98|98|96|96|97|94|99|98...,64|20|7|2|0|0|0|0|0|0|2|0|1|0|0|0|0|0|3|2|6|2|...,.,31,"30,34,30",.,31.0,31.333333,.,79,"8,42,3",.,1,0,1.0,0,3,0.0,.,.,.,0,3,0.0,1,1,0.5,3,1,0.75
REACH000067,.,,.,15,.,"7,0,11,23,55,0,2,3,4,0,0,8,46",.,.,9|14|13|23|14|22|13|13|29|12|20|2|7|5|9|18|6|4...,.,.,.,.,.,.,.,.,11,.,.,.,.,.,.,0,1,0.0,.,.,.,0,1,0.0,1,12,0.076923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
REACH000681,.,5,.,16,"-6,-8,-6,25","3,47,10,23,3,1,9,9,2,17,1,3,0,6,2,16,4,6,14,11...",.,.,0|9|59|2|56|0|0|0|0|0|0|0|0|0|6|0|0|11|0|1|9|0...,.,.,"18,17,18,28",.,.,20.25,.,.,"4,25,5,11",.,.,.,.,.,.,0,4,0.0,0,1,0.0,0,1,0.0,1,31,0.03125
REACH000682,"31,28,.,24","2,63,56,6,31,89,81,3,69,52,66,69,2,17,60,2,78,...","12,14","71,20,2,54,41,43,8,3,7,5,8,58,54,94,57,48,4,84...","30,26,-12","80,73,0,0",0|0|0|9|8|4|3|3|0|0|0|0|21|0|0|0|2|8|0|0|3|5|1...,45|97|100|100|100|100|100|22|89|17|92|100|95|1...,100|100|100|81|100|95|100|100|100|100|100|35|1...,"30,29,.,28","24,25","30,29,16",29.0,24.5,25.0,"8,2,5,3","74,86","70,8,12",0,4,0.0,2,0,1.0,1,2,0.333333,13,18,0.419355,8,20,0.285714,2,2,0.5
REACH000683,.,,.,,"31,26,23,31","2,6,4,1,1,20,7,4,33,9,19,4,1,1,0,0,6,6,3,2,1,2...",.,.,0|0|0|18|0|0|0|0|0|0|0|0|0|0|1|33|0|0|0|0|0|0|...,.,.,"30,29,28,30",.,.,29.25,.,.,"2,4,29,6",.,.,.,.,.,.,0,4,0.0,.,.,.,.,.,.,0,29,0.0
REACH000684,"30,24,24",,"23,15,24",,.,"2,4,5,7,3,0,27,1,10,4,2,6,6,0,28,1,6,12,1,87,5...",3|58|0|2|5|0|0|0|0|0|0|0|0|0|0|0|0|0|0|14|0|0|...,0|0|0|8|1|0|0|0|0|0|0|0|0|0|0|0|0|0|0|2|2|4|0|...,.,"30,28,28","28,25,28",.,28.666667,27.0,.,"2,7,3","1,13,3",.,0,3,0.0,0,3,0.0,.,.,.,.,.,.,.,.,.,2,30,0.0625


In [17]:
### Repeat length correction for some large deviation families 
### after averaging over corresponding haplotypes in children and parents for mendelian consistence
### subjects corrected: 
#REACH000519, C, F
#REACH000518, C, F
#REACH000523, M, F
#REACH000524, D, M

hp_c_h1 = (df_methyl.loc['REACH000519', 'repeat_length_h1'] + ',' + 
           df_methyl.loc['REACH000518', 'repeat_length_h1'] + ',' + 
           df_methyl.loc['REACH000524', 'repeat_length_h0']).split(',')
hp_c_h1 = np.mean([float(x) for x in hp_c_h1])
print(f'hp_c_h1: {hp_c_h1}')
df_methyl.loc['REACH000519', 'mean_repeat_length_h1'] = hp_c_h1
df_methyl.loc['REACH000518', 'mean_repeat_length_h1'] = hp_c_h1
df_methyl.loc['REACH000524', 'mean_repeat_length_h0'] = hp_c_h1

hp_c1_h2 = (df_methyl.loc['REACH000519', 'repeat_length_h2'] + ',' + 
            df_methyl.loc['REACH000523', 'repeat_length_h1']).split(',')
hp_c1_h2 = np.mean([float(x) for x in hp_c1_h2])
print(f'hp_c1_h2: {hp_c1_h2}')
df_methyl.loc['REACH000519', 'mean_repeat_length_h2'] = hp_c1_h2
df_methyl.loc['REACH000523', 'mean_repeat_length_h1'] = hp_c1_h2

hp_c2_h2 = (df_methyl.loc['REACH000518', 'repeat_length_h2'] + ',' + 
            df_methyl.loc['REACH000523', 'repeat_length_h2']).split(',')
hp_c2_h2 = np.mean([float(x) for x in hp_c2_h2])
print(f'hp_c2_h2: {hp_c2_h2}')
df_methyl.loc['REACH000518', 'mean_repeat_length_h2'] = hp_c2_h2
df_methyl.loc['REACH000523', 'mean_repeat_length_h2'] = hp_c2_h2

hp_c_h1: 35.0
hp_c1_h2: 20.833333333333332
hp_c2_h2: 29.444444444444443


In [18]:
### subjects corrected: 
#REACH000563, C, F
#REACH000560, C, M
#REACH000561, M, F
#REACH000562, D, M

hp_c_h1 = (df_methyl.loc['REACH000563', 'repeat_length_h1'] + ',' + 
           df_methyl.loc['REACH000562', 'repeat_length_h0']).split(',')
hp_c_h1 = np.mean([float(x) for x in hp_c_h1])
print(f'hp_c_h1: {hp_c_h1}')
df_methyl.loc['REACH000563', 'mean_repeat_length_h1'] = hp_c_h1
df_methyl.loc['REACH000562', 'mean_repeat_length_h0'] = hp_c_h1

hp_c1_h2 = (df_methyl.loc['REACH000563', 'repeat_length_h2'] + ',' + 
            df_methyl.loc['REACH000561', 'repeat_length_h2']).split(',')
hp_c1_h2 = np.mean([float(x) for x in hp_c1_h2])
print(f'hp_c1_h2: {hp_c1_h2}')
df_methyl.loc['REACH000563', 'mean_repeat_length_h2'] = hp_c1_h2
df_methyl.loc['REACH000561', 'mean_repeat_length_h2'] = hp_c1_h2

hp_m_h1 = (df_methyl.loc['REACH000561', 'repeat_length_h1']).split(',')
hp_m_h1 = np.mean([float(x) for x in hp_m_h1])
print(f'hp_m_h1: {hp_m_h1}')
df_methyl.loc['REACH000561', 'mean_repeat_length_h1'] = hp_m_h1

hp_c_h1: 26.375
hp_c1_h2: 49.111111111111114
hp_m_h1: 28.25


In [19]:
### subjects corrected: 
#REACH000453, C, F
#REACH000451, M, F

hp_c1_h2 = (df_methyl.loc['REACH000453', 'repeat_length_h2'] + ',' + 
            df_methyl.loc['REACH000451', 'repeat_length_h2']).split(',')
hp_c1_h2 = np.mean([float(x) for x in hp_c1_h2])
print(f'hp_c1_h2: {hp_c1_h2}')
df_methyl.loc['REACH000453', 'mean_repeat_length_h2'] = hp_c1_h2
df_methyl.loc['REACH000451', 'mean_repeat_length_h2'] = hp_c1_h2

hp_c_h1 = (df_methyl.loc['REACH000453', 'repeat_length_h1']).split(',')
hp_c_h1 = np.mean([float(x) for x in hp_c_h1])
print(f'hp_c_h1: {hp_c_h1}')
df_methyl.loc['REACH000453', 'mean_repeat_length_h1'] = hp_c_h1

hp_m_h1 = (df_methyl.loc['REACH000451', 'repeat_length_h1']).split(',')
hp_m_h1 = np.mean([float(x) for x in hp_m_h1])
print(f'hp_m_h1: {hp_m_h1}')
df_methyl.loc['REACH000451', 'mean_repeat_length_h1'] = hp_m_h1

hp_c1_h2: 49.4
hp_c_h1: 31.2
hp_m_h1: 28.8


In [22]:
### write the final table
for i in range(1,11):
    df_methyl[f'PC{i}'] = df_methyl.index.str.split('_').str[0].map(df_pca_flt.set_index('iid_orig')[f'PC{i}'])
df_methyl['sample_id'] = df_methyl.index.map(lambda x: x if 'REACH000226' not in x else 'REACH000226')
df_methyl['fid'] = df_methyl.sample_id.str.split('_').str[0].map(df_meta.set_index('Sample_ID_orig').Family_ID)
df_methyl['rel'] = df_methyl.sample_id.str.split('_').str[0].map(df_meta.set_index('Sample_ID_orig').Rel_Proband)
df_methyl['sex'] = df_methyl.sample_id.str.split('_').str[0].map(df_meta.set_index('Sample_ID_orig').Sex.apply(lambda x: 'Male' if x=='M' else 'Female'))
df_methyl['Platform'] = df_methyl.sample_id.map(df_cov.set_index('SAMPLE').COHORT) # df_cov has the sample/platform naming inside
df_methyl['coverage'] = df_methyl.sample_id.map(df_cov.set_index('SAMPLE').MEAN_COVERAGE) # df_cov has the sample/platform naming inside
df_methyl['Affected'] = df_methyl.sample_id.str.split('_').str[0].map(df_meta.set_index('Sample_ID_orig').Affected)
df_methyl['Phenotype'] = df_methyl.Affected.map(lambda x: 1 if x=='Yes' else 0)
df_methyl['in_trios'] = df_methyl.index.map(lambda x: 'TRUE' if x in df_pca_flt.iid.tolist() or x=='REACH000226_sperm' else 'FALSE')

### for those subjects with both haplotypes present (must be females), compute large and small haplotype stats (frac, mean rep, skewness)
### haplotype of col_in_1 must correspond to col_out_1 and the same for _2
### output is large: first, small: second
def get_large_small_vals(row, col_in_1, col_in_2, col_out_1, col_out_2):
    if row.sex != 'Female':
        return '.', '.'
    if row.num_methyl_reads_h1 == '.' or row.num_methyl_reads_h2 == '.':
        return '.', '.'
    if row[col_in_1] >= row[col_in_2]:
        out = (row[col_out_1], row[col_out_2])
    else:
        out = (row[col_out_2], row[col_out_1])
    return out


### figure out which hp is the large and with is the small CGG repeat
col_in_1 = 'mean_repeat_length_h1'
col_in_2 = 'mean_repeat_length_h2'
col_out_1 = 'frac_hap_methyl_h1'
col_out_2 = 'frac_hap_methyl_h2'
df_methyl[['frac_hap_methyl_large_rep', 'frac_hap_methyl_small_rep']] = df_methyl.apply(lambda row: get_large_small_vals(row, col_in_1, col_in_2, 
                                                                                                                         col_out_1, col_out_2), axis=1, result_type = 'expand')

col_in_1 = 'mean_repeat_length_h1'
col_in_2 = 'mean_repeat_length_h2'
col_out_1 = 'mean_repeat_length_h1'
col_out_2 = 'mean_repeat_length_h2'
df_methyl[['mean_large_rep', 'mean_small_rep']] = df_methyl.apply(lambda row: get_large_small_vals(row, col_in_1, col_in_2, 
                                                                                                   col_out_1, col_out_2), axis=1, result_type = 'expand')

df_methyl['has_both_hps'] = df_methyl.mean_large_rep.map(lambda x: 'TRUE' if x!='.' else 'FALSE')

def get_binomtest_pval(row):
    if row.has_both_hps == 'FALSE':
        return '.'
    k = row.num_methyl_reads_h1 + row.num_unmethyl_reads_h2
    n = row.num_methyl_reads_h1 + row.num_unmethyl_reads_h1 + row.num_methyl_reads_h2 + row.num_unmethyl_reads_h2
    return binomtest(k, n, p=0.5, alternative='two-sided').pvalue

df_methyl['binomial_pval'] = df_methyl.apply(lambda row: get_binomtest_pval(row), axis=1)
df_methyl['binomial_phred'] = df_methyl['binomial_pval'].apply(lambda x: -10*np.log10(x) if x!='.' else '.')

def get_rep_group(row):
    if row.has_both_hps == 'FALSE':
        return '.'
    out = ('≥35 CGG' if ((row.mean_repeat_length_h1 >= 35) or 
                         (row.mean_repeat_length_h2 >= 35)) else 
           ('≤25 CGG' if ((row.mean_repeat_length_h1 <= 25) or 
                          (row.mean_repeat_length_h2 <= 25)) else '26-34 CGG'))
    return out
df_methyl['repeat_group'] = df_methyl.apply(lambda row: get_rep_group(row), axis=1)

### last column: both haplotype has certain number of reads
def get_min_reads(row, num_reads_thr=2):
    if row.sex != 'Female':
        return '.'
    if row.num_methyl_reads_h1 == '.' or row.num_methyl_reads_h2 == '.':
        return '.'
    if (row.num_methyl_reads_h1 + row.num_unmethyl_reads_h1 >= num_reads_thr) and (row.num_methyl_reads_h2 + row.num_unmethyl_reads_h2 >= num_reads_thr):
        return 'TRUE'
    else:
        return 'FALSE'

num_reads_thr = 2
df_methyl['both_hps_2_reads'] = df_methyl.apply(lambda row: get_min_reads(row, num_reads_thr), axis=1)
num_reads_thr = 3
df_methyl['both_hps_3_reads'] = df_methyl.apply(lambda row: get_min_reads(row, num_reads_thr), axis=1)

print(df_methyl['in_trios'].value_counts())
print(df_methyl['Affected'].value_counts())

display(df_methyl)
### write the table
file_name = f'{dir_out}/methylation_table_complete.tsv'
print(file_name)
df_methyl.to_csv(file_name, sep='\t', index=True)

in_trios
TRUE     242
FALSE     28
Name: count, dtype: int64
Affected
No     181
Yes     89
Name: count, dtype: int64


,bp_dev_h1,xci_read_mean_methyl_lik_h1,bp_dev_h2,xci_read_mean_methyl_lik_h2,bp_dev_h0,xci_read_mean_methyl_lik_h0,methyl_h1,methyl_h2,methyl_h0,repeat_length_h1,repeat_length_h2,repeat_length_h0,mean_repeat_length_h1,mean_repeat_length_h2,mean_repeat_length_h0,read_mean_methyl_lik_h1,read_mean_methyl_lik_h2,read_mean_methyl_lik_h0,num_methyl_reads_h1,num_unmethyl_reads_h1,frac_hap_methyl_h1,num_methyl_reads_h2,num_unmethyl_reads_h2,frac_hap_methyl_h2,num_methyl_reads_h0,num_unmethyl_reads_h0,frac_hap_methyl_h0,xci_num_methyl_reads_h1,xci_num_unmethyl_reads_h1,xci_frac_hap_methyl_h1,xci_num_methyl_reads_h2,xci_num_unmethyl_reads_h2,xci_frac_hap_methyl_h2,xci_num_methyl_reads_h0,xci_num_unmethyl_reads_h0,xci_frac_hap_methyl_h0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,sample_id,fid,rel,sex,Platform,coverage,Affected,Phenotype,in_trios,frac_hap_methyl_large_rep,frac_hap_methyl_small_rep,mean_large_rep,mean_small_rep,has_both_hps,binomial_pval,binomial_phred,repeat_group,both_hps_2_reads,both_hps_3_reads
REACH000026,.,,.,,.,"22,10,10,0,2,7,4,2,20,19,20,2,18",.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,0,13,0.0,0.095364,-0.045230,-0.019472,0.155381,0.000061,-0.004595,-0.023594,0.038160,0.011138,-0.011087,REACH000026,F0026,Proband,Male,PB,3.22,Yes,1,TRUE,.,.,.,.,FALSE,.,.,.,.,.
REACH000058,.,,.,,"51,51,51,57,51,51,52","3,4,9,10,2,2,7,2,39,8,12,1,17,0,0,1,1,14,29,10...",.,.,0|0|0|0|0|0|0|1|2|32|27|13|7|5|46|27|29|15|3|4...,.,.,"37,37,37,39,37,37,37",.,.,37.285714,.,.,"8,9,11,17,23,5,17",.,.,.,.,.,.,0,7,0.0,.,.,.,.,.,.,0,28,0.0,0.002422,-0.009216,-0.007762,0.073817,0.015720,0.006303,0.055403,-0.038327,-0.011381,0.032498,REACH000058,F0058,Proband,Male,PB,6.19,Yes,1,TRUE,.,.,.,.,FALSE,.,.,.,.,.
REACH000065,.,,.,,30,"9,10",.,.,62|37|36|51|8|2|0|0|0|0|0|7|46|88|91|87|84|66|...,.,.,30,.,.,30.0,.,.,22,.,.,.,.,.,.,0,1,0.0,.,.,.,.,.,.,0,2,0.0,0.252774,0.107933,0.199378,0.002387,0.001311,0.008149,0.012198,-0.000825,-0.002550,-0.030490,REACH000065,F0065,Proband,Male,PB,1.22,Yes,1,TRUE,.,.,.,.,FALSE,.,.,.,.,.
REACH000066,32,"1,12,34","31,43,30","87,0",.,"71,77,1,88",91|95|100|99|96|100|99|98|98|96|96|97|94|99|98...,64|20|7|2|0|0|0|0|0|0|2|0|1|0|0|0|0|0|3|2|6|2|...,.,31,"30,34,30",.,31.0,31.333333,.,79,"8,42,3",.,1,0,1.0,0,3,0.0,.,.,.,0,3,0.0,1,1,0.5,3,1,0.75,0.243117,0.049799,0.158347,0.011411,0.004728,0.007690,-0.006605,0.008261,-0.012000,-0.009906,REACH000066,F0065,Mom,Female,PB,1.61,No,0,TRUE,0.0,1.0,31.333333,31.0,TRUE,0.125,9.0309,26-34 CGG,FALSE,FALSE
REACH000067,.,,.,15,.,"7,0,11,23,55,0,2,3,4,0,0,8,46",.,.,9|14|13|23|14|22|13|13|29|12|20|2|7|5|9|18|6|4...,.,.,.,.,.,.,.,.,11,.,.,.,.,.,.,0,1,0.0,.,.,.,0,1,0.0,1,12,0.076923,0.245851,0.155945,0.240343,0.002973,0.002622,-0.000228,0.004678,-0.002355,0.011371,-0.027117,REACH000067,F0065,Dad,Male,PB,1.42,No,0,TRUE,.,.,.,.,FALSE,.,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
REACH000681,.,5,.,16,"-6,-8,-6,25","3,47,10,23,3,1,9,9,2,17,1,3,0,6,2,16,4,6,14,11...",.,.,0|9|59|2|56|0|0|0|0|0|0|0|0|0|6|0|0|11|0|1|9|0...,.,.,"18,17,18,28",.,.,20.25,.,.,"4,25,5,11",.,.,.,.,.,.,0,4,0.0,0,1,0.0,0,1,0.0,1,31,0.03125,-0.030336,-0.005133,-0.003167,-0.015673,-0.001873,0.015788,0.017784,-0.002266,-0.019622,-0.022165,REACH000681,F0270,Proband,Male,ONT,10.04,Yes,1,TRUE,.,.,.,.,FALSE,.,.,.,.,.
REACH000682,"31,28,.,24","2,63,56,6,31,89,81,3,69,52,66,69,2,17,60,2,78,...","12,14","71,20,2,54,41,43,8,3,7,5,8,58,54,94,57,48,4,84...","30,26,-12","80,73,0,0",0|0|0|9|8|4|3|3|0|0|0|0|21|0|0|0|2|8|0|0|3|5|1...,45|97|100|100|100|100|100|22|89|17|92|100|95|1...,100|100|100|81|100|95|100|100|100|100|100|35|1...,"30,29,.,28","24,25","30,29,16",29.0,24.5,25.0,"8,2,5,3","74,86","70,8,12",0,4,0.0,2,0,1.0,1,2,0.333333,13,18,0.419355,8,20,0.285714,2,2,0.5,-0.022231,-0.005122,-0.006389,-0.005036,0.0044

outputs/methylation_table_complete.tsv


In [23]:
### Make a table of haplotype information for clogit/logit models
### set females:
df_methyl_f = df_methyl.loc[(df_methyl.sex=='Female')].copy()
df_methyl_f = df_methyl_f.loc[(df_methyl_f.num_methyl_reads_h1!='.') | (df_methyl_f.num_methyl_reads_h2!='.')]
df_methyl_f['num_reads_h1'] = df_methyl_f.apply(lambda row: int(row.num_methyl_reads_h1) + int(row.num_unmethyl_reads_h1) if row.num_methyl_reads_h1!='.' else 0, axis=1)
df_methyl_f['num_reads_h2'] = df_methyl_f.apply(lambda row: int(row.num_methyl_reads_h2) + int(row.num_unmethyl_reads_h2) if row.num_methyl_reads_h2!='.' else 0, axis=1)
df_methyl_f['xci_num_reads_h1'] = df_methyl_f.apply(lambda row: int(row.xci_num_methyl_reads_h1) + int(row.xci_num_unmethyl_reads_h1) if row.xci_num_methyl_reads_h1!='.' else 0, axis=1)
df_methyl_f['xci_num_reads_h2'] = df_methyl_f.apply(lambda row: int(row.xci_num_methyl_reads_h2) + int(row.xci_num_unmethyl_reads_h2) if row.xci_num_methyl_reads_h2!='.' else 0, axis=1)
df_methyl_f = df_methyl_f.loc[(df_methyl_f.num_reads_h1>=3) | (df_methyl_f.num_reads_h2>=3)]

df_out_f = pd.DataFrame()
for idx, row in df_methyl_f.iterrows():
    sample = idx
    for hp in ['h1', 'h2']:
        if row[f'num_reads_{hp}'] >= 3:
            mean_rep = row[f'mean_repeat_length_{hp}']
            frac_hap = row[f'frac_hap_methyl_{hp}']
            xci_frac_hap = row[f'xci_frac_hap_methyl_{hp}']
            num_reads = row[f'num_reads_{hp}']
            xci_num_reads = row[f'xci_num_reads_{hp}']
            rep_group = row['repeat_group']
            df_tmp = pd.DataFrame({'subject_id': [idx], 'sex': ['Female'], 'haplotype': [hp], 
                                   f'mean_repeat_length': [mean_rep], f'repeat_group': [rep_group], 
                                   f'frac_hap_methyl': [frac_hap], f'xci_frac_hap_methyl': [xci_frac_hap], 
                                   f'num_reads': [num_reads], f'xci_num_reads': [xci_num_reads]})
            df_out_f = pd.concat([df_out_f, df_tmp], ignore_index=True)

### also get rid of the haplotypes where xci_frac_hap_methyl are missing.
df_out_f = df_out_f.loc[df_out_f.xci_frac_hap_methyl!='.']

tot_cols = ['subject_id', 'sex', 'haplotype', 'mean_repeat_length', 'repeat_group', 
            'frac_hap_methyl', 'xci_frac_hap_methyl', 'num_reads', 'xci_num_reads']

# set male subjects
df_methyl_m = df_methyl.loc[(df_methyl.sex=='Male')].copy()
df_methyl_m = df_methyl_m.loc[(df_methyl_m.num_methyl_reads_h1!='.') | (df_methyl_m.num_methyl_reads_h2!='.') | (df_methyl_m.num_methyl_reads_h0!='.')]
df_methyl_m['repeat_lengths'] = df_methyl_m.apply(lambda row: ','.join([x for x in [row.repeat_length_h1, row.repeat_length_h2, row.repeat_length_h0] if x!='.']), axis=1)
df_methyl_m['mean_repeat_length'] = df_methyl_m.repeat_lengths.map(lambda x: np.mean([int(xx) for xx in x.split(',') if xx!='.' and xx!='']))
df_methyl_m['num_methyl_reads'] = df_methyl_m.apply(lambda row: np.sum([int(x) for x in [row.num_methyl_reads_h1, row.num_methyl_reads_h2, row.num_methyl_reads_h0] if x!='.']), axis=1)
df_methyl_m['num_unmethyl_reads'] = df_methyl_m.apply(lambda row: np.sum([int(x) for x in [row.num_unmethyl_reads_h1, row.num_unmethyl_reads_h2, row.num_unmethyl_reads_h0] if x!='.']), axis=1)
df_methyl_m['num_reads'] = df_methyl_m['num_methyl_reads'] + df_methyl_m['num_unmethyl_reads']
df_methyl_m['frac_hap_methyl'] = df_methyl_m['num_methyl_reads'] / (df_methyl_m['num_methyl_reads'] + df_methyl_m['num_unmethyl_reads'])
df_methyl_m['xci_num_methyl_reads'] = df_methyl_m.apply(lambda row: np.sum([int(x) for x in [row.xci_num_methyl_reads_h1, row.xci_num_methyl_reads_h2, row.xci_num_methyl_reads_h0] if x!='.']), axis=1)
df_methyl_m['xci_num_unmethyl_reads'] = df_methyl_m.apply(lambda row: np.sum([int(x) for x in [row.xci_num_unmethyl_reads_h1, row.xci_num_unmethyl_reads_h2, row.xci_num_unmethyl_reads_h0] if x!='.']), axis=1)
df_methyl_m['xci_num_reads'] = df_methyl_m['xci_num_methyl_reads'] + df_methyl_m['xci_num_unmethyl_reads']
df_methyl_m['xci_frac_hap_methyl'] = df_methyl_m['xci_num_methyl_reads'] / (df_methyl_m['xci_num_methyl_reads'] + df_methyl_m['xci_num_unmethyl_reads'])
df_methyl_m['subject_id'] = df_methyl_m.index
df_methyl_m['haplotype'] = 'hm'

df_out_m = df_methyl_m.loc[df_methyl_m.num_reads>=4][tot_cols]

### concatinate female and male
df_out = pd.concat([df_out_f, df_out_m], ignore_index=True)
df_out['subject_id'] = df_out.subject_id.apply(lambda x: 'REACH000226' if x == 'REACH000226_sperm' else x)
for i in range(1,11):
    df_out[f'PC{i}'] = df_out.subject_id.str.split('_').str[0].map(df_pca_flt.set_index('iid_orig')[f'PC{i}'])
df_out['fid'] = df_out.subject_id.str.split('_').str[0].map(df_meta.set_index('Sample_ID_orig').Family_ID)
df_out['rel'] = df_out.subject_id.str.split('_').str[0].map(df_meta.set_index('Sample_ID_orig').Rel_Proband)
df_out['Platform'] = df_out.subject_id.map(df_cov.set_index('SAMPLE').COHORT) # df_cov has the sample/platform naming inside
df_out['coverage'] = df_out.subject_id.map(df_cov.set_index('SAMPLE').MEAN_COVERAGE) # df_cov has the sample/platform naming inside
df_out['Affected'] = df_out.subject_id.str.split('_').str[0].map(df_meta.set_index('Sample_ID_orig').Affected)
df_out['Phenotype'] = df_out.Affected.map(lambda x: 1 if x=='Yes' else 0)
df_out['in_trios'] = df_out.subject_id.map(lambda x: 'TRUE' if x in df_pca_flt.iid.tolist() or x=='REACH000226_sperm' else 'FALSE')

cols = ['mean_repeat_length', 'frac_hap_methyl', 'xci_frac_hap_methyl', 'xci_num_reads', 'num_reads']
for col in cols:
    df_out[col] = df_out[col].astype(float)

def get_rep_group2(row):
    if row.repeat_group != '.':
        out = row.repeat_group
    else:
        out = ('≥35 CGG' if (row.mean_repeat_length >= 35) else 
               ('≤25 CGG' if (row.mean_repeat_length <= 25) else '26-34 CGG'))
    return out
df_out['repeat_group'] = df_out.apply(lambda row: get_rep_group2(row), axis=1)

print('df_out:')
display(df_out)

### write the table
file_name = f'{dir_out}/methylation_table_complete_hps.tsv'
print(file_name)
df_out.to_csv(file_name, sep='\t', index=False)

df_out:


/home/smmortazavi/micromamba/envs/bioinf/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/smmortazavi/micromamba/envs/bioinf/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


,subject_id,sex,haplotype,mean_repeat_length,repeat_group,frac_hap_methyl,xci_frac_hap_methyl,num_reads,xci_num_reads,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,fid,rel,Platform,coverage,Affected,Phenotype,in_trios
0,REACH000066,Female,h2,31.333333,26-34 CGG,0.000000,0.500000,3.0,2.0,0.243117,0.049799,0.158347,0.011411,0.004728,0.007690,-0.006605,0.008261,-0.012000,-0.009906,F0065,Mom,PB,1.61,No,0,TRUE
1,REACH000087,Female,h2,29.500000,26-34 CGG,0.750000,0.666667,4.0,9.0,0.129968,0.080387,0.132056,0.029583,0.020156,0.007971,-0.002383,0.014468,0.006253,-0.018303,F0078,Mom,PB,2.75,No,0,TRUE
2,REACH000089,Female,h2,30.333333,26-34 CGG,0.666667,1.000000,3.0,3.0,0.049880,0.047204,0.069405,0.015002,0.011334,0.011519,-0.001072,0.014418,-0.017609,-0.005676,F0078,Sibling,PB,3.27,Yes,1,TRUE
3,REACH000092,Female,h2,29.800000,26-34 CGG,0.600000,0.777778,5.0,9.0,-0.028511,-0.001002,-0.001921,0.004426,-0.001891,0.006074,0.027474,0.004611,0.015596,0.030866,F0081,Proband,PB,4.89,Yes,1,TRUE
4,REACH000093,Female,h1,29.250000,26-34 CGG,1.000000,0.277778,4.0,18.0,-0.027082,0.003555,-0.003174,-0.000206,-0.004555,0.000972,-0.004064,-0.011024,0.005814,-0.009496,F0081,Mom,PB,4.63,No,0,TRUE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,REACH000663,Male,hm,33.166667,26-34 CGG,0.000000,0.025000,6.0,40.0,0.464341,0.385613,-0.153238,-0.033060,0.029758,0.012759,0.018026,0.006101,-0.000833,-0.016008,F0266,Sibling,ONT,10.64,No,0,TRUE
129,REACH000681,Male,hm,20.250000,≤25 CGG,0.000000,0.029412,4.0,34.0,-0.030336,-0.005133,-0.003167,-0.015673,-0.001873,0.015788,0.017784,-0.002266,-0.019622,-0.022165,F0270,Proband,ONT,10.04,Yes,1,TRUE
130,REACH000683,Male,hm,29.250000,26-34 CGG,0.000000,0.000000,4.0,29.0,-0.029616,-0.004138,0.000015,-0.011085,0.006907,0.004983,0.000198,-0.006297,0.002731,-0.031329,F0270,Dad,ONT,9.97,No,0,TRUE
131,REACH000684,Male,hm,27.833333,26-34 CGG,0.000000,0.062500,6.0,32.0,-0.021597,-0.007737,-0.005243,-0.009369,0.003756,0.019086,0.016933,-0.002215,-0.021031,-0.017064,F0270,Sibling,ONT,7.84,No,0,TRUE


outputs/methylation_table_complete_hps.tsv


In [ ]:
### make a table of raw data for other select genes

In [24]:
### read select genes methylation for subjects in the skew plot
file_in_og = f'{dir_data}/methyl_data_select_genes.tsv'
df_og = pd.read_table(file_in_og, sep='\t', header=0, keep_default_na=False)

pd.set_option('display.max_columns', None)
print('df_og:')
display(df_og)
pd.set_option('display.max_columns', 10)

file_sam = f'{dir_data}/methyl_data_sample_order_select_genes.txt'
with open(file_sam, 'r') as fh:
    sample_order_og = fh.readline().split(',')[:-1] # last one is an empty string
print(f'number of samples: {len(sample_order_og)}')
#print(f'sample order: {sample_order_og}')

df_exnd_og = df_og[['chrom' ,'pos', 'end', 'region_id']].copy()
cols = df_og.columns.tolist()[4:]
#print(cols)

for idx, sample in enumerate(sample_order_og):
    for col in cols:
        df_exnd_og[f'{sample}_{col}'] = df_og[col].str.split(';').apply(lambda row: row[idx])
print('df_exnd_og:')
display(df_exnd_og)

df_og:


,chrom,pos,end,region_id,methyl_h1,methyl_h2,methyl_h0,bp_dev_h1,bp_dev_h2,bp_dev_h0,rn_h1,rn_h2,rn_h0,ps_h1,ps_h2
0,chrX,10620123,10620805,MID1_5UTR,.;.;.;.;.;.;.;.;.;2|2|34|21|9|38|52|66|56|18|1...,.;.;.;.;.;.;.;.;.;22|55|23|34|52|71|37|63|36|3...,2|2|11|4|0|2|33|10|1|0|1|1|1|0|1|4|11|3|5|1|0|...,".;.;.;.;.;.;.;.;.;0;.;.;.;.;.;.;.;-3,.;.;.;.;....",".;.;.;.;.;.;.;.;.;3;.;.;1;.;.;.;.;-6,1;.;.;.;....","0,.;-1,4;1;8,2,3,0;.;8;4,1,0,4;4,0;-4,0;15;-1,...",.;.;.;.;.;.;.;.;.;m54340U_210709_004131/129893...,.;.;.;.;.;.;.;.;.;m54340U_210709_004131/139856...,"m54340U_200716_001732/101845162/ccs,m54340U_20...",".;.;.;.;.;.;.;.;.;10185;.;.;.;.;.;.;.;10185,10...",.;.;.;.;.;.;.;.;.;10185;.;.;10185;.;.;.;.;1018...
1,chrX,11663929,11665723,ARHGAP6_5UTR,.;.;.;12|33|72|92|98|99|98|99|96|93|95|99|99|9...,.;.;.;16|14|19|79|98|80|90|52|57|28|4|5|4|9|24...,99|98|96|94|88|27|58|48|25|20|5|5|2|1|0|0|1|3|...,".;.;.;3;.;.;.;.;0;0;.,0,0,24,0,.;.;.;.;-2;.;.;...",".;.;.;8;.;.;.;.;.;0,25;16,17,1,0,.;.;.;.;2,7,6...","20,15;2,5,7,.;.;.;19,-6;.;.,1,14,-2,1;.,0,2;-1...",.;.;.;m54340U_200716_153006/176228322/ccs;.;.;...,.;.;.;m54340U_200716_153006/53151007/ccs;.;.;....,"m54340U_200716_001732/98502494/ccs,m54340U_200...",".;.;.;10185;.;.;.;.;10215;10185;10185,10185,10...",".;.;.;10185;.;.;.;.;.;10185,10185;10185,10185,..."
2,chrX,12791015,12791585,PRPS2_5UTR,.;.;.;96|82|42|36|92|94|91|74|94|99|99|97|97|9...,.;.;.;.;.;.;.;.;1|0|0|0|0|0|0|0|0|7|15|19|8|4|...,0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|2|8|5|7|2|...,".;.;.;1;.;.;-1,2;.;-1;0,0;-5;.;7,5;.;8,0;.;.;....",".;.;.;.;.;.;.;.;-3;0,-2,0;.;.;0,0,-3,6,2,-1;.;...","1,-2,1;0,0,-2,2,5;.;.;5;3,1,1;.;0,1,1,10,.;.;....",.;.;.;m54340U_200716_153006/120785681/ccs;.;.;...,.;.;.;.;.;.;.;.;m54340U_210703_193517/36570698...,"m54340U_200716_001732/45089815/ccs,m54340U_200...",".;.;.;10185;.;.;10215,10215;.;10215;10185,1018...",".;.;.;.;.;.;.;.;10215;10185,10185,10185;.;.;10..."
3,chrX,13044401,13045071,FAM9C_5UTR,.;.;.;46|53|92|91|65|93|98|99|100|100|100|100|...,.;.;.;74|88|95|96|96|97|94|98|98|93|95|95|83|9...,94|90|88|72|76|79|29|41|98|98|100|100|99|100|1...,".;.;.;5,5;.;.;1;.;.;-1,3,1;.;.;1;.;.;.;.;.;.;....",".;.;.;5,1;.;.;.;.;-2;2,-1,.;.;.;2,8;.;.;.;.;2,...","3,2,0,3;2,2,12,.;.;.;-3;0,2,2,1,0;0,-2,4,-1,0;...",".;.;.;m54340U_200716_153006/162006345/ccs,m543...",".;.;.;m54340U_200716_153006/13370548/ccs,m5434...","m54340U_200716_001732/18612485/ccs,m54340U_200...",".;.;.;10185,10185;.;.;10215;.;.;10185,10185,10...",".;.;.;10185,10185;.;.;.;.;10215;10185,10185,10..."
4,chrX,13816674,13817842,GPM6B_5UTR,.;.;.;21|34|70|55|72|63|57|59|24|7|15|13|9|7|8...,.;.;.;26|35|9|0|0|0|0|2|2|1|2|12|20|16|2|0|1|4...,100|99|100|100|99|99|100|99|99|99|99|99|100|99...,".;.;.;-1,3,3,5,.;.;.;.;.;1;-3,7,.;-3,0,-2;.;.,...",".;.;.;9,-4;.;-2,1;2;4;6,-5;-5,.,.,0,1,12,-2,0;...","0,2;-7,2,4,-3,.;1,7;.;.;.;.;2,.,-3;.;.;.;-6,.,...",".;.;.;m54340U_200716_153006/29294712/ccs,m5434...",".;.;.;m54340U_200716_153006/110494252/ccs,m543...","m54340U_200716_001732/148963558/ccs,m54340U_20...",".;.;.;10185,10185,10185,10185,10185;.;.;.;.;10...",".;.;.;10185,10185;.;10215,10215;10215;10215;10..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,chrX,154457737,154458815,PLXNA3_5UTR,.;.;.;.;.;.;.;.;.;.;11|31|14|14|55|30|36|4|3|2...,.;.;.;.;.;.;.;.;.;4|0|0|0|0|0|0|0|0|0|5|20|20|...,0|0|0|0|18|6|1|0|2|5|1|0|0|0|0|0|0|0|0|0|0|0|0...,".;.;.;.;.;.;.;.;.;.;6,10,0;.;-1;.;.;.;.;.;.;.;...",".;.;.;.;.;.;.;.;.;6,4,1;5,16;.;7,0;.;.;.;.;.;....","5;-1,1;0;16;.;-4;0,1;0,.;2,-3;-1,.,0;.;2;.;.,3...",.;.;.;.;.;.;.;.;.;.;m54340U_210709_004131/5721...,.;.;.;.;.;.;.;.;.;m54340U_200611_224317/482370...,m54340U_200716_001732/6359572/ccs;m54340U_2105...,".;.;.;.;.;.;.;.;.;.;10185,10185,10185;.;10185;...",".;.;.;.;.;.;.;.;.;10185,10185,10185;10185,1018..."
159,chrX,154515851,154516567,FAM3A_5UTR,.;.;0|0|0|1|1|8|9|14|3|1|2|1|0|0|0|2|2|2|5|7|1...,.;.;.;.;.;.;6|0|3|16|15|23|13|11|36|68|27|9|19...,0|2|4|6|11|30|7|8|11|5|1|0|0|0|0|0|0|1|5|1|0|0...,".;.;6;6;.;.;-1,2;.;.;.;.;.;.;.;5

number of samples: 270
df_exnd_og:


,chrom,pos,end,region_id,REACH000026_methyl_h1,...,REACH000685_rn_h1,REACH000685_rn_h2,REACH000685_rn_h0,REACH000685_ps_h1,REACH000685_ps_h2
0,chrX,10620123,10620805,MID1_5UTR,.,...,.,.,"d8c3a034-713f-45e7-afc3-5bd725037e60,4ed0dc63-...",.,.
1,chrX,11663929,11665723,ARHGAP6_5UTR,.,...,.,.,"48fde9cd-8d16-4466-968f-bc2702649e69,72cf700b-...",.,.
2,chrX,12791015,12791585,PRPS2_5UTR,.,...,.,.,"7ac125ba-e89e-4916-b232-f2de4a0176b2,74c8650b-...",.,.
3,chrX,13044401,13045071,FAM9C_5UTR,.,...,.,.,"7b324b3e-8553-4a6b-a86f-218833ae9489,d050936f-...",.,.
4,chrX,13816674,13817842,GPM6B_5UTR,.,...,.,.,"e8d3ece7-d441-4bee-99fa-322791acda4d,441af2cd-...",.,.
...,...,...,...,...,...,...,...,...,...,...,...
158,chrX,154457737,154458815,PLXNA3_5UTR,.,...,.,.,"206d001e-a23b-4bc8-b79f-7dfbe1ead309,cb5fa621-...",.,.
159,chrX,154515851,154516567,FAM3A_5UTR,.,...,.,.,"efb7f9d6-7497-4cf0-ac98-7c3012616b7a,b03822db-...",.,.
160,chrX,154750529,154751289,GAB3_5UTR,.,...,.,.,"129c6e04-18e0-4f22-af8e-c5ce9d3cfd01,37594a25-...",.,.
161,chrX,154762543,154763011,DKC1_5UTR,.,...,.,.,"eb986a78-64c7-4ab1-b42e-566131904deb,44593820-...",.,.


In [25]:
def get_gene_methyl_data(region_id, df_exnd, sample_order, samples_only=None):
    df_out = pd.DataFrame()
    
    this_idx = df_exnd.loc[df_exnd.region_id==region_id].index.tolist()[0]
    cols_met = [f'methyl_h1', f'methyl_h2', f'methyl_h0']
    cols_ps = [f'ps_h1', f'ps_h2']
    for col in cols_met+cols_ps:
        df_out[col] = pd.Series(dtype='str')

    if samples_only == None:
        for sample in sample_order:        
            ### set methylation columns
            for col in cols_met:
                df_out.loc[sample, col] = df_exnd.loc[df_exnd.region_id==region_id].loc[this_idx, f'{sample}_{col}']
            ### set phase set columns
            for col in cols_ps:
                df_out.loc[sample, col] = df_exnd.loc[df_exnd.region_id==region_id].loc[this_idx, f'{sample}_{col}']
    else:
        for sample in sample_order:
            if sample in samples_only:
                ### set methylation columns
                for col in cols_met:
                    df_out.loc[sample, col] = df_exnd.loc[df_exnd.region_id==region_id].loc[this_idx, f'{sample}_{col}']
                ### set phase set columns
                for col in cols_ps:
                    df_out.loc[sample, col] = df_exnd.loc[df_exnd.region_id==region_id].loc[this_idx, f'{sample}_{col}']
    
    ### compute mean methylation likelihood for each read
    for col in cols_met:
        df_out[f'read_mean_methyl_lik_{col.split("_")[-1]}'] = df_out.apply(lambda row: ','.join([str(round(np.mean([float(xx) for xx in x.split('|')]))) for x in row[col].split(',') if x != '.'])
                                                                                 if row[col] != '.' else '.', axis=1)
    for col in cols_met:
        hp = col.split("_")[-1]
        df_out[f'num_methyl_reads_{hp}'] = df_out[f'read_mean_methyl_lik_{hp}'].apply(lambda x: len([xx for xx in x.split(',') if float(xx)>=50]) if x != '.' else '.')
        df_out[f'num_unmethyl_reads_{hp}'] = df_out[f'read_mean_methyl_lik_{hp}'].apply(lambda x: len([xx for xx in x.split(',') if float(xx)<50]) if x != '.' else '.')
        df_out[f'frac_hap_methyl_{hp}'] = df_out.apply(lambda row: int(row[f'num_methyl_reads_{hp}'])/(int(row[f'num_methyl_reads_{hp}'])+int(row[f'num_unmethyl_reads_{hp}'])) 
                                                               if (row[f'num_methyl_reads_{hp}']!='.' and row[f'num_unmethyl_reads_{hp}']!='.') else '.', axis=1)
    
    return df_out

In [26]:
### Make methylation data table of all regions for selected subjects
samples_only = ['REACH000561', 'REACH000453', 'REACH000451', 'REACH000479', 'REACH000518', 'REACH000519', 
                'REACH000234', 'REACH000365', 'REACH000436_PB', 'REACH000616', 'REACH000628', 'REACH000672', 'REACH000674', 
                'REACH000702', 'REACH000107', 'REACH000404', 'REACH000417', 'REACH000435', 'REACH000482', 'REACH000510', 
                'REACH000547', 'REACH000581', 'REACH000220']
df_methyl_sample = pd.DataFrame()
### this loop may take some time to run...
for i, region_id in enumerate(df_exnd_og.region_id.tolist()):
    if (i % 10 == 1):
        print(f'{i} {region_id}')
    this_df = get_gene_methyl_data(region_id, df_exnd_og, sample_order_og, samples_only)
    this_df['region_id'] = region_id
    df_methyl_sample = pd.concat([df_methyl_sample, this_df])

### correct FMR1 methylation data for REACH000561 from before corrections
cols_corr = ['read_mean_methyl_lik_h1', 'read_mean_methyl_lik_h2', 'read_mean_methyl_lik_h0', 
             'num_methyl_reads_h1', 'num_methyl_reads_h2', 'num_methyl_reads_h0', 
             'num_unmethyl_reads_h1', 'num_unmethyl_reads_h2', 'num_unmethyl_reads_h0', 
             'frac_hap_methyl_h1', 'frac_hap_methyl_h2', 'frac_hap_methyl_h0']
as_type = [str, str, str, int, int, str, int, int, str, float, float, str]
for col, t in zip(cols_corr, as_type):
    df_methyl_sample.loc[((df_methyl_sample.region_id=='FMR1_5UTR') & 
                          (df_methyl_sample.index=='REACH000561')), [col]] = df_methyl.loc[['REACH000561'],[col]].astype(t)

df_methyl_sample['num_reads_h1'] = df_methyl_sample.apply(lambda row: int(row.num_methyl_reads_h1) + int(row.num_unmethyl_reads_h1) 
                                                  if row.num_methyl_reads_h1!='.' else 0, axis=1)
df_methyl_sample['num_reads_h2'] = df_methyl_sample.apply(lambda row: int(row.num_methyl_reads_h2) + int(row.num_unmethyl_reads_h2) 
                                                  if row.num_methyl_reads_h2!='.' else 0, axis=1)
df_methyl_sample['Read count'] = df_methyl_sample['num_reads_h1'] + df_methyl_sample['num_reads_h2']
pd.set_option('display.max_columns', None)

pd.set_option('display.max_columns', None)
print('df_methyl_sample:')
print(df_methyl_sample.shape)
display(df_methyl_sample)
pd.set_option('display.max_columns', 10)

1 ARHGAP6_5UTR
11 PRDX4_5UTR
21 TAB3_5UTR
31 CASK_5UTR
41 RBM3_5UTR
51 PHF8_5UTR
61 EDA_5UTR
71 SLC16A2_5UTR
81 HNRNPH2_5UTR
91 BEX2_5UTR
101 PSMD10_5UTR
111 NKAP_5UTR
121 UTP14A_5UTR
131 RTL8B_5UTR
141 FMR1_5UTR
151 ZNF185_5UTR
161 DKC1_5UTR
df_methyl_sample:
(3749, 21)


,methyl_h1,methyl_h2,methyl_h0,ps_h1,ps_h2,read_mean_methyl_lik_h1,read_mean_methyl_lik_h2,read_mean_methyl_lik_h0,num_methyl_reads_h1,num_unmethyl_reads_h1,frac_hap_methyl_h1,num_methyl_reads_h2,num_unmethyl_reads_h2,frac_hap_methyl_h2,num_methyl_reads_h0,num_unmethyl_reads_h0,frac_hap_methyl_h0,region_id,num_reads_h1,num_reads_h2,Read count
REACH000220,.,.,10|12|16|13|4|5|2|4|1|0|0|0|0|0|0|0|1|0|0|0|0|...,.,.,.,.,"2,44,2,22,51,1",.,.,.,.,.,.,1,5,0.166667,MID1_5UTR,0,0,0
REACH000234,77|81|91|72|65|59|19|2|0|1|2|0|2|21|28|53|51|7...,0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|...,78|99|99|100|99|97|98|95|26|5|1|10|12|5|29|26|...,"10185,10185","10185,10185,10185","32,2","0,51,9","52,7,7,26,32,73",0,2,0.0,1,2,0.333333,2,4,0.333333,MID1_5UTR,2,3,5
REACH000365,.,.,0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|...,.,.,.,.,"0,72,0,73,1,0,2,11",.,.,.,.,.,.,2,6,0.25,MID1_5UTR,0,0,0
REACH000436_PB,44|93|96|98|97|100|99|98|73|9|2|3|0|11|66|68|7...,98|99|98|93|96|84|38|4|0|0|0|1|4|9|14|81|95|85...,.,"10532334,10532334,10532334,10532334,10532334","10532334,10532334,10532334","39,9,55,2,67","50,0,48",.,2,3,0.4,1,2,0.333333,.,.,.,MID1_5UTR,5,3,8
REACH000451,.,0|0|0|0|0|9|2|0|0|0|0|0|0|0|0|0|1|1|1|0|0|0|0|...,82|95|98|97|82|82|62|72|33|14|32|56|29|5|3|4|5...,.,10185,.,0,"53,2,43,0,1,53,79,0,89",.,.,.,0,1,0.0,4,5,0.444444,MID1_5UTR,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
REACH000518,0|0|1|0|0|0|13|0|7|11|0|0|0|6|0|0|0|0|0|0|0|4|...,1|10|4|75|1|0|1|0|0|0|1|1|60|0|0|8|2|0|0|0|0|1...,0|100|14|98|98|98|0|100|96|0|100|100|0|5|0|1|1...,"10185,10185,10185,10185","10185,10185","3,4,61,13","8,12","32,5,10,7,70,7,17,2",1,3,0.25,0,2,0.0,1,7,0.125,RAB39B_5UTR,4,2,6
REACH000519,7|0|3|0|0|0|6|41|0|0|0|3|0|0|0|0|1|0|0|0|0|2|0...,1|68|41|2|1|0|79|82|0|64|4|2|23|0|0|5|0|3|2|0|...,100|53|0|0|0|0|3|14|0|0|2|8|0|40|0|2|1|0|29|0|...,10185,10185,2,17,"8,17,53,45,52,11,2,59,16,11,29,55",0,1,0.0,0,1,0.0,4,8,0.333333,RAB39B_5UTR,1,1,2
REACH000547,33|0|5|23|59|96|3|98|0|2|30|14|18|1|16|38|11|5...,100|55|92|100|99|45|86|99|99|91|99|100|100|100...,0|0|1|29|8|0|0,"12790,12790,12790,12790,12790,12790,12790","12790,12790,12790","16,36,64,47,2,9,75","73,2,22",5,2,5,0.285714,1,2,0.333333,0,1,0.0,RAB39B_5UTR,7,3,10
REACH000561,.,.,7|65|0|0|0|0|7|0|0|0|0|0|0|0|9|0|0|0|0|1|2|57|...,.,.,.,.,"8,44,60,80,76,81,58,59,52,47,12",.,.,.,.,.,.,7,4,0.636364,RAB39B_5UTR,0,0,0


In [27]:
### write methylation data table for the 163 genes for the selected subjects
df_methyl_sample['fmr1_mean_repeat_length_h1'] = df_methyl_sample.index.map(df_methyl.mean_repeat_length_h1).astype(float)
df_methyl_sample['fmr1_mean_repeat_length_h2'] = df_methyl_sample.index.map(df_methyl.mean_repeat_length_h2).astype(float)
df_methyl_sample['fmr1_mean_repeat_length_h0'] = df_methyl_sample.index.map(df_methyl.mean_repeat_length_h0)

### for those subjects with both haplotypes present (must be females), compute large and small haplotype stats (frac, mean rep, skewness)
### haplotype of col_in_1 must correspond to col_out_1 and the same for _2
### output is large: first, small: second
def get_large_small_vals(row, col_in_1, col_in_2, col_out_1, col_out_2):
    ### no need to check for sex and '.' present in num_methyla_reads
    if row[col_in_1] >= row[col_in_2]:
        out = (row[col_out_1], row[col_out_2])
    else:
        out = (row[col_out_2], row[col_out_1])
    return out
    
### figure out which hp is the large and with is the small CGG repeat
col_in_1 = 'fmr1_mean_repeat_length_h1'
col_in_2 = 'fmr1_mean_repeat_length_h2'
col_out_1 = 'frac_hap_methyl_h1'
col_out_2 = 'frac_hap_methyl_h2'
df_methyl_sample[['frac_hap_methyl_large_rep', 'frac_hap_methyl_small_rep']] = df_methyl_sample.apply(lambda row: get_large_small_vals(row, col_in_1, col_in_2, 
                                                                                                                         col_out_1, col_out_2), axis=1, result_type = 'expand')

col_in_1 = 'fmr1_mean_repeat_length_h1'
col_in_2 = 'fmr1_mean_repeat_length_h2'
col_out_1 = 'fmr1_mean_repeat_length_h1'
col_out_2 = 'fmr1_mean_repeat_length_h2'
df_methyl_sample[['fmr1_mean_large_rep', 'fmr1_mean_small_rep']] = df_methyl_sample.apply(lambda row: get_large_small_vals(row, col_in_1, col_in_2, 
                                                                                                   col_out_1, col_out_2), axis=1, result_type = 'expand')

def get_rep_group2(row):
    out = ('≥35 CGG' if (row.fmr1_mean_repeat_length_h1 >= 35 or row.fmr1_mean_repeat_length_h2 >= 35) else 
           ('≤25 CGG' if (row.fmr1_mean_repeat_length_h1 <= 25 or row.fmr1_mean_repeat_length_h2 <= 25) else '26-34 CGG'))
    return out
df_methyl_sample['repeat_group'] = df_methyl_sample.apply(lambda row: get_rep_group2(row), axis=1)

pd.set_option('display.max_columns', None)
print('df_methyl_sample:')
display(df_methyl_sample)
pd.set_option('display.max_columns', 10)

### write the table
file_name = f'{dir_out}/methylation_table_select_genes.tsv'
print(file_name)
df_methyl_sample.to_csv(file_name, sep='\t', index=True)

df_methyl_sample:


,methyl_h1,methyl_h2,methyl_h0,ps_h1,ps_h2,read_mean_methyl_lik_h1,read_mean_methyl_lik_h2,read_mean_methyl_lik_h0,num_methyl_reads_h1,num_unmethyl_reads_h1,frac_hap_methyl_h1,num_methyl_reads_h2,num_unmethyl_reads_h2,frac_hap_methyl_h2,num_methyl_reads_h0,num_unmethyl_reads_h0,frac_hap_methyl_h0,region_id,num_reads_h1,num_reads_h2,Read count,fmr1_mean_repeat_length_h1,fmr1_mean_repeat_length_h2,fmr1_mean_repeat_length_h0,frac_hap_methyl_large_rep,frac_hap_methyl_small_rep,fmr1_mean_large_rep,fmr1_mean_small_rep,repeat_group
REACH000220,.,.,10|12|16|13|4|5|2|4|1|0|0|0|0|0|0|0|1|0|0|0|0|...,.,.,.,.,"2,44,2,22,51,1",.,.,.,.,.,.,1,5,0.166667,MID1_5UTR,0,0,0,29.000000,29.000000,.,.,.,29.000000,29.000000,26-34 CGG
REACH000234,77|81|91|72|65|59|19|2|0|1|2|0|2|21|28|53|51|7...,0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|...,78|99|99|100|99|97|98|95|26|5|1|10|12|5|29|26|...,"10185,10185","10185,10185,10185","32,2","0,51,9","52,7,7,26,32,73",0,2,0.0,1,2,0.333333,2,4,0.333333,MID1_5UTR,2,3,5,29.333333,30.600000,.,0.333333,0.0,30.600000,29.333333,26-34 CGG
REACH000365,.,.,0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|...,.,.,.,.,"0,72,0,73,1,0,2,11",.,.,.,.,.,.,2,6,0.25,MID1_5UTR,0,0,0,30.166667,29.250000,.,.,.,30.166667,29.250000,26-34 CGG
REACH000436_PB,44|93|96|98|97|100|99|98|73|9|2|3|0|11|66|68|7...,98|99|98|93|96|84|38|4|0|0|0|1|4|9|14|81|95|85...,.,"10532334,10532334,10532334,10532334,10532334","10532334,10532334,10532334","39,9,55,2,67","50,0,48",.,2,3,0.4,1,2,0.333333,.,.,.,MID1_5UTR,5,3,8,23.750000,29.833333,.,0.333333,0.4,29.833333,23.750000,≤25 CGG
REACH000451,.,0|0|0|0|0|9|2|0|0|0|0|0|0|0|0|0|1|1|1|0|0|0|0|...,82|95|98|97|82|82|62|72|33|14|32|56|29|5|3|4|5...,.,10185,.,0,"53,2,43,0,1,53,79,0,89",.,.,.,0,1,0.0,4,5,0.444444,MID1_5UTR,0,1,1,28.800000,49.400000,.,0.0,.,49.400000,28.800000,≥35 CGG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
REACH000518,0|0|1|0|0|0|13|0|7|11|0|0|0|6|0|0|0|0|0|0|0|4|...,1|10|4|75|1|0|1|0|0|0|1|1|60|0|0|8|2|0|0|0|0|1...,0|100|14|98|98|98|0|100|96|0|100|100|0|5|0|1|1...,"10185,10185,10185,10185","10185,10185","3,4,61,13","8,12","32,5,10,7,70,7,17,2",1,3,0.25,0,2,0.0,1,7,0.125,RAB39B_5UTR,4,2,6,35.000000,29.444444,28.5,0.25,0.0,35.000000,29.444444,≥35 CGG
REACH000519,7|0|3|0|0|0|6|41|0|0|0|3|0|0|0|0|1|0|0|0|0|2|0...,1|68|41|2|1|0|79|82|0|64|4|2|23|0|0|5|0|3|2|0|...,100|53|0|0|0|0|3|14|0|0|2|8|0|40|0|2|1|0|29|0|...,10185,10185,2,17,"8,17,53,45,52,11,2,59,16,11,29,55",0,1,0.0,0,1,0.0,4,8,0.333333,RAB39B_5UTR,1,1,2,35.000000,20.833333,27.0,0.0,0.0,35.000000,20.833333,≥35 CGG
REACH000547,33|0|5|23|59|96|3|98|0|2|30|14|18|1|16|38|11|5...,100|55|92|100|99|45|86|99|99|91|99|100|100|100...,0|0|1|29|8|0|0,"12790,12790,12790,12790,12790,12790,12790","12790,12790,12790","16,36,64,47,2,9,75","73,2,22",5,2,5,0.285714,1,2,0.333333,0,1,0.0,RAB39B_5UTR,7,3,10,28.000000,33.750000,.,0.333333,0.285714,33.750000,28.000000,26-34 CGG
REACH000561,.,.,7|65|0|0|0|0|7|0|0|0|0|0|0|0|9|0|0|0|0|1|2|57|...,.,.,.,.,"8,44,60,80,76,81,58,59,52,47,12",.,.,.,.,.,.,7,4,0.636364,RAB39B_5UTR,0,0,0,28.250000,49.111111,.,.,.,49.111111,28.250000,≥35 CGG


outputs/methylation_table_select_genes.tsv
